In [1]:
import os

In [2]:
%pwd

'c:\\Users\\yitayew\\Desktop\\redwinequality\\End-to-end-ml-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\yitayew\\Desktop\\redwinequality\\End-to-end-ml-project'

In [5]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path          # Import the Path class for handling file system paths


@dataclass(frozen=True)  # Define a dataclass with immutable fields
class DataIngestionConfig:
    # Attributes of the DataIngestionConfig class with their types
    root_dir: Path           # Directory where data ingestion artifacts will be stored
    source_URL: str          # URL from which the data will be downloaded
    local_data_file: Path    # Path to the local file where downloaded data will be saved
    unzip_dir: Path          # Directory where the downloaded data will be unzipped

In [10]:
from src.mlproject import*
from src.mlproject.utils.common import read_yaml,create_directories

ModuleNotFoundError: No module named 'mlproject'

In [8]:
class ConfigurationManager:
    # Constructor method to initialize the ConfigurationManager class
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Path to the configuration file, with a default value
        params_filepath=PARAMS_FILE_PATH,  # Path to the parameters file, with a default value
        schema_filepath=SCHEMA_FILE_PATH   # Path to the schema file, with a default value
    ):
        # Reading and loading the configuration, parameters, and schema files
        self.config = read_yaml(config_filepath)  # Load configuration settings from the YAML file
        self.params = read_yaml(params_filepath)  # Load parameters from the YAML file
        self.schema = read_yaml(schema_filepath)  # Load schema definitions from the YAML file

        # Create directories specified in the configuration for storing artifacts
        create_directories([self.config.artifacts_root])

    # Method to get the data ingestion configuration
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion  # Access the data ingestion section from the config

        # Create the root directory for data ingestion if it doesn't already exist
        create_directories([config.root_dir])

        # Create a DataIngestionConfig object using values from the configuration file
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,              # Root directory for data ingestion
            source_URL=config.source_URL,          # URL to download the data from
            local_data_file=config.local_data_file,  # Path to the local data file
            unzip_dir=config.unzip_dir             # Directory where the data will be unzipped
        )

        # Return the data ingestion configuration object
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size

ModuleNotFoundError: No module named 'mlproject'

In [ ]:
#pip install PyYAML

In [ ]:
class DataIngestion:
    # Constructor method to initialize the DataIngestion class
    def __init__(self, config: DataIngestionConfig):
        self.config = config  # Store the configuration object for data ingestion

    # Method to download the file from the specified URL
    def download_file(self):
        # Check if the file already exists at the local_data_file path
        if not os.path.exists(self.config.local_data_file):
            # If the file doesn't exist, download it from the source URL and save it locally
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,             # URL to download the file from
                filename=self.config.local_data_file    # Local path to save the downloaded file
            )
            # Log a message indicating that the file was successfully downloaded
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            # If the file already exists, log a message indicating its size
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # Method to extract a zip file to the specified directory
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        Function returns None.
        """
        unzip_path = self.config.unzip_dir  # Path where the zip file will be extracted
        os.makedirs(unzip_path, exist_ok=True)  # Create the unzip directory if it doesn't exist
        # Open the zip file and extract all its contents to the unzip path
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  # Extract all files to the specified directory

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e